# Import Libararies

In [1]:
import pandas as pd
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox

# Read Data From User

In [10]:
processes = []
used_names = set()  # To store used process names

def add_process():
    name = name_entry.get().strip()  # Remove leading/trailing whitespace
    arrival_time = get_entry_as_int(arrival_entry)
    execution_time = get_entry_as_int(execution_entry)
    queue_number = get_entry_as_int(queue_entry)

    if not name or arrival_time is None or execution_time is None or queue_number is None:
        show_error("Please fill in all fields with valid data.")
        return

    if name in used_names:
        show_error("Name already exists. Enter a unique name.")
        return

    if queue_number not in [0, 1, 2]:
        show_error("Invalid queue number. Process not added.")
        return

    process = {
        "name": name,
        "arrival_time": arrival_time,
        "execution_time": execution_time,
        "queue_number": queue_number
    }

    processes.append(process)
    used_names.add(name)
    clear_entries()
    show_info("Process added successfully.")

def get_entry_as_int(entry):
    value = entry.get()
    if value.strip() == '':
        return None
    try:
        return int(value)
    except ValueError:
        return None

def show_error(message):
    messagebox.showerror("Error", message)

def show_info(message):
    messagebox.showinfo("Success", message)

def clear_entries():
    for entry in [name_entry, arrival_entry, execution_entry, queue_entry]:
        entry.delete(0, tk.END)

def display_processes():
    # Create a new window for displaying processes
    display_window = tk.Toplevel()
    display_window.title("Processes")

    # Create a Text widget to display processes
    text_area = tk.Text(display_window, height=20, width=80)
    text_area.pack()

    # Insert each process into the Text widget
    for process in processes:
        text_area.insert(tk.END, f"Process: {process['name']} Arrival: {process['arrival_time']} "
                         f"Execution: {process['execution_time']} Queue: {process['queue_number']}\n")
        
    text_area.configure(state="disabled")  # Disable editing

    # Run the new window
    display_window.mainloop()

def create_gui():
    global name_entry, arrival_entry, execution_entry, queue_entry  # Declare as global

    root = tk.Tk()
    root.title("Process Details")
    root.geometry("600x200")

    # Styling labels
    label_font = ("Arial", 10)
    entry_padding_x = 10
    entry_padding_y = 5
    labels = ["Name:", "Arrival Time:", "Execution Time:", "Queue Number (0, 1, 2):"]
    
    for i, label_text in enumerate(labels):
        label = tk.Label(root, text=label_text, font=label_font,padx=entry_padding_x, pady=entry_padding_y)
        label.grid(row=i, column=0)

    # Styling entry widgets
    entry_font = ("Arial", 10)
    entry_bg_color = "white"
    entry_widgets = [tk.Entry(root, font=entry_font, bg=entry_bg_color) for _ in range(len(labels))]
    for i, entry_widget in enumerate(entry_widgets):
        entry_widget.grid(row=i, column=1,padx=entry_padding_x, pady=entry_padding_y)
    
    # Assign each entry to the corresponding global variable
    name_entry = entry_widgets[0]
    arrival_entry = entry_widgets[1]
    execution_entry = entry_widgets[2]
    queue_entry = entry_widgets[3]

    # Buttons
    add_button = tk.Button(root, text="Add Process", command=add_process, font=("Arial", 10), bg="#4CAF50",
                           cursor="hand2",bd=0,relief=tk.FLAT, fg="#FFFFFF")
    add_button.grid(row=5, column=0, padx=0, pady=10)

    display_button = tk.Button(root,text="Display Processes",command=display_processes,font=("Arial", 10),
                               bg="#03A9F4",cursor="hand2",bd=0, relief=tk.FLAT, fg="#FFFFFF")
    display_button.grid(row=5, column=1, padx=25, pady=10)

    clear_button = tk.Button(root, text="Clear Entries", command=clear_entries, font=("Arial", 10), bg="#03A9F4",
                             cursor="hand2", bd=0, relief=tk.FLAT, fg="#FFFFFF")
    clear_button.grid(row=5, column=2, padx=25, pady=10)

    # Exit button
    exit_button = tk.Button(root, text="Exit", command=root.destroy, font=("Arial", 10), bg="#F4511E",
                            padx=10,cursor="hand2", bd=0, relief=tk.FLAT, fg="#FFFFFF")
    exit_button.grid(row=5, column=3, padx=25, pady=10)
    
    root.mainloop()

create_gui()

In [11]:
# processes = [
#     {'name': 'p0', 'arrival_time': 0, 'execution_time': 5, 'queue_number': 2},
#     {'name': 'p1', 'arrival_time': 0, 'execution_time': 3, 'queue_number': 1},
#     {'name': 'p2', 'arrival_time': 0, 'execution_time': 1, 'queue_number': 0},
#     {'name': 'p3', 'arrival_time': 10, 'execution_time': 6, 'queue_number': 1},
#     {'name': 'p4', 'arrival_time': 12, 'execution_time': 8, 'queue_number': 2},
#     {'name': 'p5', 'arrival_time': 14, 'execution_time': 5, 'queue_number': 0},
#     {'name': 'p6', 'arrival_time': 15, 'execution_time': 2, 'queue_number': 0},
#     {'name': 'p7', 'arrival_time': 17, 'execution_time': 1, 'queue_number': 1},
#     {'name': 'p8', 'arrival_time': 20, 'execution_time': 7, 'queue_number': 2}
# ]

In [12]:
df_p =pd.DataFrame(processes)
gantt_chart = []  # To store Gantt chart entries
time = 0
rr_q=3
srt_q=2
# Initialize queues directly from the DataFrame based on conditions
HighQueue = df_p[(df_p['queue_number'] == 0) & (df_p['arrival_time'] == time)].to_dict('records')
MediumQueue = df_p[(df_p['queue_number'] == 1) & (df_p['arrival_time'] == time)].to_dict('records')
LowQueue = df_p[(df_p['queue_number'] == 2) & (df_p['arrival_time'] == time)].to_dict('records')
all_processes = df_p.to_dict('records')
# Initialize lists to store process names entering each queue
high_queue_copy = []
medium_queue_copy = []
low_queue_copy = []
AllData = df_p.to_dict('records')
AllData = [p for p in processes if p['arrival_time'] != time ]

In [13]:
while HighQueue or MediumQueue or LowQueue or AllData :
    for process in HighQueue:
        if  process['execution_time'] > 0  :
                run_time = min(rr_q, process['execution_time'])
                gantt_chart.append({'process': process['name'], 'start_time': time, 'end_time': time + run_time})
                print(f"{process['name']} Entered the kernel for {run_time} units")
                process['execution_time'] -= run_time
                high_queue_copy.append(process)
                
                for unit in range(0,run_time):
                    time += 1
                    NewProcess = df_p.loc[df_p.arrival_time == time].to_dict('records')
                    if NewProcess:
                        for new_process in NewProcess:
                            if new_process['queue_number'] == 0:
                                HighQueue.append(new_process)
                                print(f'process {new_process["name"]} entered the High Queue at time {time}')
                            elif new_process['queue_number'] == 1: 
                                MediumQueue.append(new_process)
                                print(f'process {new_process["name"]} entered the Medium Queue at time {time}')
                            else:  
                                LowQueue.append(new_process)
                                print(f'process {new_process["name"]} entered the Low Queue at time {time}')
                                
                if process['execution_time'] <= 0:
                    print(f"{process['name']} has finished execution at {time} ")
                    process['turnaround_time'] = time - process['arrival_time']
                    index_to_update = df_p.index[df_p['name'] == process['name']].tolist()[0]
                    df_p.at[index_to_update, 'turnaround_time'] = process['turnaround_time']
                    HighQueue.remove(process)
    
    for process in MediumQueue:
        if HighQueue:
            break
        if process['execution_time'] > 0 :
            process = min(MediumQueue, key=lambda x: x['execution_time'])
            run_time = min(srt_q, process['execution_time'])
            gantt_chart.append({'process': process['name'], 'start_time': time, 'end_time': time + run_time})
            print(f"Running {process['name']} Entered the kernel for {run_time} units")
            process['execution_time'] -= run_time
            medium_queue_copy.append(process)
            
            for _ in range(run_time):
                time += 1
                NewProcess = df_p.loc[df_p.arrival_time == time].to_dict('records')
                if NewProcess:
                    for new_process in NewProcess:
                        if new_process['queue_number'] == 0:
                            print(f'process {new_process["name"]} entered the High Queue at time {time}')
                            HighQueue.append(new_process)
                        elif new_process['queue_number'] == 1:
                            MediumQueue.append(new_process)
                            print(f'process {new_process["name"]} entered the Medium Queue at time {time}')                                
                        else:
                            LowQueue.append(new_process)
                            print(f'process {new_process["name"]} entered the Low Queue at time {time}')
        
            if process['execution_time'] <= 0:
                print(f"{process['name']} has finished execution at {time} ")
                process['turnaround_time'] = time - process['arrival_time']
                index_to_update = df_p.index[df_p['name'] == process['name']].tolist()[0]
                df_p.at[index_to_update, 'turnaround_time'] = process['turnaround_time']
                MediumQueue.remove(process)  
    
    for process in LowQueue:
        if HighQueue or MediumQueue:
            break
        if process['execution_time'] > 0:
            process = min(LowQueue, key=lambda x: x['execution_time'])
            run_time = process['execution_time']
            gantt_chart.append({'process': process['name'], 'start_time': time, 'end_time': time + run_time})
            print(f"{process['name']} Entered the kernel for {run_time} units")
            process['execution_time'] -= run_time
            low_queue_copy.append(process)
            
            for _ in range(run_time):
                time += 1
                NewProcess = []
                NewProcess.extend(df_p.loc[df_p.arrival_time == time].to_dict('records'))
                if len(NewProcess) > 0:
                    for new_process in NewProcess:
                        if new_process['queue_number'] == 0:
                            HighQueue.append(new_process)
                            print(f'process {new_process["name"]} entered the High Queue at time {time}')
                        elif new_process['queue_number'] == 1:
                            MediumQueue.append(new_process)
                            print(f'process {new_process["name"]} entered the Medium Queue at time {time}')
                        else:
                            LowQueue.append(new_process)
                            print(f'process {new_process["name"]} entered the Low Queue at time {time}')
        
            if process['execution_time'] <= 0:
                print(f"{process['name']} has finished execution at {time} ")
                process['turnaround_time'] = time - process['arrival_time']
                index_to_update = df_p.index[df_p['name'] == process['name']].tolist()[0]
                df_p.at[index_to_update, 'turnaround_time'] = process['turnaround_time']
                LowQueue.remove(process)

    while not HighQueue and not MediumQueue and not LowQueue and AllData  :
        AllData = [p for p in AllData if p['arrival_time'] > time]
        if AllData:
            time += 1
            new_processes = [p for p in AllData if p['arrival_time'] == time]
            for new_process in new_processes:
                new_processes.remove(new_process)
                if new_process['queue_number'] == 0:
                    HighQueue.append(new_process)
                    print(f'process {new_process["name"]} entered the High Queue at time {time}')
                elif new_process['queue_number'] == 1:
                    MediumQueue.append(new_process)
                    print(f'process {new_process["name"]} entered the Medium Queue at time {time}')
                else:
                    LowQueue.append(new_process)
                    print(f'process {new_process["name"]} entered the Low Queue at time {time}')

p2 Entered the kernel for 1 units
p2 has finished execution at 1 
Running p1 Entered the kernel for 2 units
Running p1 Entered the kernel for 1 units
p1 has finished execution at 4 
p0 Entered the kernel for 5 units
p0 has finished execution at 9 
process p3 entered the Medium Queue at time 10
Running p3 Entered the kernel for 2 units
process p4 entered the Low Queue at time 12
Running p3 Entered the kernel for 2 units
process p5 entered the High Queue at time 14
p5 Entered the kernel for 3 units
process p6 entered the High Queue at time 15
process p7 entered the Medium Queue at time 17
p6 Entered the kernel for 2 units
p6 has finished execution at 19 
p5 Entered the kernel for 2 units
process p8 entered the Low Queue at time 20
p5 has finished execution at 21 
Running p7 Entered the kernel for 1 units
p7 has finished execution at 22 
Running p3 Entered the kernel for 2 units
p3 has finished execution at 24 
p8 Entered the kernel for 7 units
p8 has finished execution at 31 
p4 Entered 

In [14]:
high_queue_copy = [entry['name'] for entry in high_queue_copy]
queue_string = ', '.join(high_queue_copy)
print(f"ready queue for high queue: {queue_string}")
medium_queue_copy = [entry['name'] for entry in medium_queue_copy]
queue_string = ', '.join(medium_queue_copy)
print(f"ready queue for medium queue: {queue_string}")
low_queue_copy = [entry['name'] for entry in low_queue_copy]
queue_string = ', '.join(low_queue_copy)
print(f"ready queue for low queue: {queue_string}")

ready queue for high queue: p2, p5, p6, p5
ready queue for medium queue: p1, p1, p3, p3, p7, p3
ready queue for low queue: p0, p8, p4


In [15]:
# Display Gantt chart
print("\nGantt Chart:")
print("Process\tStart Time\tEnd Time")
for entry in gantt_chart:
    print(f"{entry['process']}\t{entry['start_time']}\t\t{entry['end_time']}")


Gantt Chart:
Process	Start Time	End Time
p2	0		1
p1	1		3
p1	3		4
p0	4		9
p3	10		12
p3	12		14
p5	14		17
p6	17		19
p5	19		21
p7	21		22
p3	22		24
p8	24		31
p4	31		39


In [16]:
df_p['waiting_time'] = df_p['turnaround_time'] - df_p['execution_time']
# Convert columns to integer type
df_p['waiting_time'] = df_p['waiting_time'].astype(int)
df_p['turnaround_time'] = df_p['turnaround_time'].astype(int)
df_p

,name,arrival_time,execution_time,queue_number,turnaround_time,waiting_time
0,p0,0,5,2,9,4
1,p1,0,3,1,4,1
2,p2,0,1,0,1,0
3,p3,10,6,1,14,8
4,p4,12,8,2,27,19
5,p5,14,5,0,7,2
6,p6,15,2,0,4,2
7,p7,17,1,1,5,4
8,p8,20,7,2,11,4


In [17]:
Avg_waiting = sum(df_p['waiting_time'])/len(df_p)
print('Averge waiting time : ',Avg_waiting)

Averge waiting time :  4.888888888888889


In [18]:
def create_GUI(gantt_chart, high_queue_copy, medium_queue_copy, low_queue_copy, df_p, Avg_waiting):
    root = tk.Tk()
    root.title("Gantt Chart / Ready Queues")
    
    canvas_width = 1300
    canvas_height = 600
    canvas = tk.Canvas(root, width=canvas_width, height=canvas_height)
    canvas.pack()
    
    rect_width = 100
    rect_height = 30
    
    # Displaying Gantt Chart
    y_offset = 50
    for i, process in enumerate(gantt_chart):
        name = process['process']
        start = process['start_time']
        end = process['end_time']
    
        x_start = start * 30 + 100
        x_end = end * 30 + 100
    
        canvas.create_oval(x_start, y_offset, x_end, y_offset + rect_height, fill='lightblue')
        canvas.create_text((x_start + x_end) / 2, y_offset + rect_height / 2, text=name)
    
        canvas.create_text(x_start, y_offset + rect_height + 5, anchor='w', text=f"{start}")
        if i == len(gantt_chart) - 1:  # For the last rectangle
            canvas.create_text(x_end, y_offset + rect_height + 5, anchor='e', text=f"{end}")
        elif i < len(gantt_chart) - 1 and end != gantt_chart[i + 1]['start_time']:
            canvas.create_text(x_end, y_offset + rect_height + 5, anchor='e', text=f"{end}")
    
    # Label above the Gantt Chart
    canvas.create_text(canvas_width / 2, y_offset - 20, text="Gantt Chart", font=('Arial', 14, 'bold'))
    
    # Adding content visualization for high_queue_copy, medium_queue_copy, and low_queue_copy
    queue_labels = ['High Queue', 'Medium Queue', 'Low Queue']
    y_offset = 200  # Starting y_offset for the queues
    
    for i, queue in enumerate([high_queue_copy, medium_queue_copy, low_queue_copy]):
        x_offset = (canvas_width - rect_width * len(queue)) / 2  # Centering horizontally
    
        # Display queue labels
        canvas.create_text(canvas_width / 2, y_offset - 20, text=queue_labels[i], font=('Arial', 12, 'bold'))
        
        for j, process_name in enumerate(queue):
            name = process_name  
            
            canvas.create_rectangle(x_offset, y_offset, x_offset + rect_width, y_offset + rect_height, fill='lightgreen')
            canvas.create_text(x_offset + rect_width / 2, y_offset + rect_height / 2, text=name)
            
            x_offset += rect_width  # Move to the next position
    
        y_offset += 150  # Increment vertical offset for the next queue
    
    
    # Creating a new window to display DataFrame content and average waiting time
    df_window = tk.Toplevel()
    df_window.title("Processes Waiting Time")
    
    # Frame to hold DataFrame content
    df_frame = tk.Frame(df_window)
    df_frame.pack(padx=20, pady=20)  # Padding added here
    
    # Label for DataFrame content
    df_label = ttk.Label(df_frame, text="Process Details", font=('Arial', 12, 'bold'))
    df_label.grid(row=0, column=0, pady=10, columnspan=3)
    
    # Creating a Treeview widget for DataFrame content
    df_tree = ttk.Treeview(df_frame, columns=list(df_p.columns), show="headings")
    df_tree.grid(row=1, column=0, columnspan=3)
    
    # Define columns for the Treeview
    for col in df_p.columns:
        df_tree.heading(col, text=col)
        df_tree.column(col, width=100)
    
    # Inserting DataFrame content into the Treeview
    for idx, row in df_p.iterrows():
        df_tree.insert("", "end", values=list(row))
    
    # Label for Average Waiting Time
    avg_label = ttk.Label(df_frame, text=f"Avg Waiting Time: {Avg_waiting}", font=('Arial', 12, 'bold'))
    avg_label.grid(row=2, column=0, pady=10, columnspan=3)


    root.mainloop()

# Call the function with the provided arguments
create_GUI(gantt_chart, high_queue_copy, medium_queue_copy, low_queue_copy, df_p, Avg_waiting)